In [1]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.9.1
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [2]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

In [3]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))

Training entries: 25000, test entries: 25000


In [4]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

In [5]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,705
Trainable params: 48,191,705
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [7]:
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [8]:
r = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/40
30/30 [==============================] - 14s 443ms/step - loss: 0.6704 - accuracy: 0.5981 - val_loss: 0.6148 - val_accuracy: 0.6985
Epoch 2/40
30/30 [==============================] - 13s 436ms/step - loss: 0.5399 - accuracy: 0.7697 - val_loss: 0.4818 - val_accuracy: 0.7977
Epoch 3/40
30/30 [==============================] - 13s 438ms/step - loss: 0.3682 - accuracy: 0.8662 - val_loss: 0.3625 - val_accuracy: 0.8490
Epoch 4/40
30/30 [==============================] - 13s 438ms/step - loss: 0.2399 - accuracy: 0.9191 - val_loss: 0.3194 - val_accuracy: 0.8621
Epoch 5/40
30/30 [==============================] - 13s 438ms/step - loss: 0.1597 - accuracy: 0.9509 - val_loss: 0.3025 - val_accuracy: 0.8714
Epoch 6/40
30/30 [==============================] - 13s 433ms/step - loss: 0.1040 - accuracy: 0.9722 - val_loss: 0.3082 - val_accuracy: 0.8747
Epoch 7/40
30/30 [==============================] - 13s 436ms/step - loss: 0.0656 - accuracy: 0.9864 - val_loss: 0.3238 - val_accuracy: 0.8726